In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

## **Grid Search ARIMA Model Hyperparameters**

The ARIMA model for time series analysis and forecasting can be tricky to configure. There are 3 parameters that require estimation by iterative trial and error from reviewing diagnostic plots and using 40-year-old heuristic rules. We can automate the process of evaluating a large number of hyperparameters for the ARIMA model by using a grid search procedure.

**Grid Searching Method**

We can automate the process of training and evaluating ARIMA models on different combinations of 
model hyperparameters. In machine learning this is called a grid search or model tuning.

The approach is broken down into two parts:
1. Evaluate an ARIMA model.
2. Evaluate sets of ARIMA parameters.

**Evaluate ARIMA Model**

This approach involves the following steps:
1. Split the dataset into training and test sets.
2. Walk the time steps in the test dataset.

(a) Train an ARIMA model.

(b) Make a one-step prediction.

(c) Store prediction; get and store actual observation.

3. Calculate error score for predictions compared to expected values. 

We can implement this in Python as a new standalone function called evaluate arima model() that takes a time series dataset as input as well as a tuple with the p, d, and q parameters for the model to be evaluated. The dataset is split in two: 66% for the initial training dataset and the remaining 34% for the test dataset.

Each time step of the test set is iterated. Just one iteration provides a model that you could use to make predictions on new data. The iterative approach allows a new ARIMA model to be trained each time step. A prediction is made each iteration and stored in a list. This is so that at the end of the test set, all predictions can be compared to the list of expected values and an error score calculated. In this case, a root mean squared error score is calculated and returned. The complete function is listed below.

```
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
# prepare training dataset
		train_size = int(len(X) * 0.66)
		train, test = X[0:train_size], X[train_size:]
		history = [x for x in train]
		# make predictions
		predictions = list()
		for t in range(len(test)):
				model = ARIMA(history, order=arima_order)
				model_fit = model.fit(disp=0)
				yhat = model_fit.forecast()[0]
				predictions.append(yhat)
				history.append(test[t])
		# calculate out-of-sample error
		rmse = sqrt(mean_squared_error(test, predictions))
		return rmse
```

**Iterate ARIMA Parameters**

Evaluating a suite of parameters is relatively straightforward. The user must specify a grid of p, d, and q ARIMA parameters to iterate. A model is created for each parameter and its performance evaluated by calling the evaluate arima model() function described in the previous section. The function must keep track of the lowest error score observed and the configuration that caused it. This can be summarized at the end of the function with a print to standard out.

We can implement this function called evaluate models() as a series of four loops. There are two additional considerations. The first is to ensure the input data are floating point values (as opposed to integers or strings), as this can cause the ARIMA procedure to fail. Second, the Statsmodels ARIMA procedure internally uses numerical optimization procedures to find a set of coefficients for the model.

```
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    rmse = evaluate_arima_model(dataset, order)
                    if rmse < best_score:
                        best_score, best_cfg = rmse, order
                    print(f"ARIMA: {order}, RMSE={rmse}")
                except:
                    continue
    print(f'Best ARIMA: {best_cfg}, RMSE={best_score}')
```

**Shampoo Sales Case Study**



In [2]:
# load dataset
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
def parser(x):
    return pd.datetime.strptime('190'+x, "%Y-%m")
series = pd.read_csv('monthly-shampoo-sales.csv', header=0,parse_dates=True,index_col=0,
                    squeeze=True,date_parser=parser)

series.head()

Month
1901-01-01    266.0
1901-02-01    145.9
1901-03-01    183.1
1901-04-01    119.3
1901-05-01    180.3
Name: Sales, dtype: float64

In [3]:
# evaluate parameters
# p_values = [0,1,2,4,6,8,10]
p_values = [0,1,2,4]
d_values = range(0,3)
q_values = range(0,3)

In [4]:
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
# prepare training dataset
		train_size = int(len(X) * 0.66)
		train, test = X[0:train_size], X[train_size:]
		history = [x for x in train]
		# make predictions
		predictions = list()
		for t in range(len(test)):
				model = ARIMA(history, order=arima_order)
				model_fit = model.fit()
				yhat = model_fit.forecast()[0]
				predictions.append(yhat)
				history.append(test[t])
		# calculate out-of-sample error
		rmse = mean_squared_error(test, predictions,squared=False)
		return rmse

In [5]:
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    rmse = evaluate_arima_model(dataset, order)
                    if rmse < best_score:
                        best_score, best_cfg = rmse, order
                    print(f"ARIMA: {order}, RMSE={rmse}")
                except:
                    continue
    print(f'Best ARIMA: {best_cfg}, RMSE={best_score}')

In [7]:
warnings.filterwarnings("ignore")
evaluate_models(series.values, p_values, d_values, q_values)

ARIMA: (0, 0, 0), RMSE=228.96565352403775
ARIMA: (0, 0, 1), RMSE=195.59598621855295
ARIMA: (0, 0, 2), RMSE=154.8860176073436
ARIMA: (0, 1, 0), RMSE=133.15599277594362
ARIMA: (0, 1, 1), RMSE=104.07674196291818
ARIMA: (0, 1, 2), RMSE=68.34454730581291
ARIMA: (0, 2, 0), RMSE=255.18668879980098
ARIMA: (0, 2, 1), RMSE=134.16842354768858
ARIMA: (0, 2, 2), RMSE=74.64390442018576
ARIMA: (1, 0, 0), RMSE=152.02793967870213
ARIMA: (1, 0, 1), RMSE=111.78718805439284
ARIMA: (1, 0, 2), RMSE=77.09175525253376
ARIMA: (1, 1, 0), RMSE=88.63093913273268
ARIMA: (1, 1, 1), RMSE=87.94241760457055
ARIMA: (1, 1, 2), RMSE=90.98639663497354
ARIMA: (1, 2, 0), RMSE=134.5762140550146
ARIMA: (1, 2, 1), RMSE=86.15734597117228
ARIMA: (1, 2, 2), RMSE=65.51057554367522
ARIMA: (2, 0, 0), RMSE=100.87910704781405
ARIMA: (2, 0, 1), RMSE=98.95301124039884
ARIMA: (2, 0, 2), RMSE=98.68871394878457
ARIMA: (2, 1, 0), RMSE=85.06349333500745
ARIMA: (2, 1, 1), RMSE=88.42791741742784
ARIMA: (2, 1, 2), RMSE=83.49919657770857
ARIMA: 

**Daily Female Births Case Study**

In [8]:
series = pd.read_csv('daily-total-female-births.csv', header=0,
                    parse_dates=True,index_col=0,squeeze=True)

In [9]:
p_values = [0,1,2,4,6,8,10]
d_values = range(0,3)
q_values = range(0,3)

In [10]:
warnings.filterwarnings("ignore")
evaluate_models(series.values, p_values, d_values, q_values)

ARIMA: (0, 0, 0), RMSE=8.189193460669514
ARIMA: (0, 0, 1), RMSE=7.884453496406967
ARIMA: (0, 0, 2), RMSE=7.7707096084433775
ARIMA: (0, 1, 0), RMSE=9.151174787971215
ARIMA: (0, 1, 1), RMSE=7.426989634243819
ARIMA: (0, 1, 2), RMSE=7.351722389371844
ARIMA: (0, 2, 0), RMSE=15.669588380043999
ARIMA: (0, 2, 1), RMSE=9.167337087616598
ARIMA: (0, 2, 2), RMSE=7.45403131397974
ARIMA: (1, 0, 0), RMSE=7.802234156510252
ARIMA: (1, 0, 1), RMSE=7.567944935270391
ARIMA: (1, 0, 2), RMSE=7.550569315971624
ARIMA: (1, 1, 0), RMSE=8.105754582175601
ARIMA: (1, 1, 1), RMSE=7.340454870932896
ARIMA: (1, 1, 2), RMSE=7.329386102593364
ARIMA: (1, 2, 0), RMSE=11.968457146549
ARIMA: (1, 2, 1), RMSE=8.11984964088242
ARIMA: (1, 2, 2), RMSE=7.407060530976554
ARIMA: (2, 0, 0), RMSE=7.697471622159684
ARIMA: (2, 0, 1), RMSE=7.537903567707994
ARIMA: (2, 1, 0), RMSE=7.700077851465145
ARIMA: (2, 1, 1), RMSE=7.332344510157906
ARIMA: (2, 1, 2), RMSE=7.355466466691162
ARIMA: (2, 2, 0), RMSE=10.3546795557865
ARIMA: (2, 2, 1), R

**Extensions**

The grid search method used in this tutorial is simple and can easily be extended. This section lists some ideas to extend the approach you may wish to explore.

- Seed Grid. The classical diagnostic tools of ACF and PACF plots can still be used with the results used to seed the grid of ARIMA parameters to search.
- Alternate Measures. The search seeks to optimize the out-of-sample root mean squared error. This could be changed to another out-of-sample statistic, an in-sample statistic, such as AIC or BIC, or some combination of the two. You can choose a metric that is most meaningful on your project.
- Residual Diagnostics. Statistics can automatically be calculated on the residual forecast errors to provide an additional indication of the quality of the fit. Examples include statistical tests for whether the distribution of residuals is Gaussian and whether there is an autocorrelation in the residuals.
- Update Model. The ARIMA model is created from scratch for each one-step forecast. With careful inspection of the API, it may be possible to update the internal data of the model with new observations rather than recreating it from scratch.
- Preconditions. The ARIMA model can make assumptions about the time series dataset, such as normality and stationarity. These could be checked and a warning raised for a given of a dataset prior to a given model being trained.